In [1]:
!pip install transformers
!python -m pip install git+https://github.com/huggingface/optimum.git
!pip install onnxruntime
!pip install onnx

  Using cached transformers-4.21.1-py3-none-any.whl (4.7 MB)
     |████████████████████████████████| 6.6 MB 2.2 MB/s eta 0:00:01
     |████████████████████████████████| 765 kB 82 kB/s eta 0:00:011
     |████████████████████████████████| 661 kB 693 kB/s eta 0:00:01
     |████████████████████████████████| 101 kB 769 kB/s ta 0:00:011
     |████████████████████████████████| 17.1 MB 1.5 MB/s eta 0:00:01
  Using cached typing_extensions-4.3.0-py3-none-any.whl (25 kB)
  Cloning https://github.com/huggingface/optimum.git to /tmp/pip-req-build-8esgx8d0
  Running command git clone -q https://github.com/huggingface/optimum.git /tmp/pip-req-build-8esgx8d0
  Resolved https://github.com/huggingface/optimum.git to commit 69f2883d945ccc669bdd0c78e0ba8fe4265c6792
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 776.4 MB 20 kB/s  eta 0:00:014   |████                            | 98.5 MB 1

# Benchmarking

## Feature Extraction

### Base Model Load





In [2]:
from transformers import AutoTokenizer, pipeline

model_id = "microsoft/codebert-base"
task = "feature-extraction"

tokenizer = AutoTokenizer.from_pretrained(model_id)

# test the model with using transformers pipeline, with handle_impossible_answer for squad_v2 
feature_extractor = pipeline(task, model=model_id, tokenizer=tokenizer, handle_impossible_answer=True)


/home/humza/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
%%time
prediction = feature_extractor("What's my name?")

CPU times: user 408 ms, sys: 51.8 ms, total: 460 ms
Wall time: 476 ms


### Optimized Model

In [5]:
from pathlib import Path
from transformers import AutoTokenizer, pipeline
from optimum.onnxruntime import ORTModelForFeatureExtraction

model_id = "microsoft/codebert-base"
onnx_path = Path("onnx")
task = "feature-extraction"

# load vanilla transformers and convert to onnx
model = ORTModelForFeatureExtraction.from_pretrained(model_id, from_transformers=True)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# save onnx checkpoint and tokenizer
model.save_pretrained(onnx_path)
tokenizer.save_pretrained(onnx_path)

# test the model with using transformers pipeline, with handle_impossible_answer for squad_v2
optimum_feature_extractor = pipeline(task, model=model, tokenizer=tokenizer, handle_impossible_answer=True)

Downloading: 100%|██████████| 498/498 [00:00<00:00, 320kB/s]


In [2]:
!pip install datasets

     |████████████████████████████████| 365 kB 482 kB/s eta 0:00:01
     |████████████████████████████████| 132 kB 7.0 MB/s eta 0:00:01
     |████████████████████████████████| 141 kB 7.4 MB/s eta 0:00:01
     |████████████████████████████████| 211 kB 7.1 MB/s eta 0:00:01
     |████████████████████████████████| 35.3 MB 295 kB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 253 kB/s eta 0:00:01
     |████████████████████████████████| 158 kB 286 kB/s eta 0:00:01
     |████████████████████████████████| 114 kB 154 kB/s eta 0:00:01
     |████████████████████████████████| 264 kB 151 kB/s eta 0:00:01


In [7]:
from pathlib import Path

from optimum.onnxruntime import ORTOptimizer
from optimum.onnxruntime.configuration import OptimizationConfig

model_id = "microsoft/codebert-base"
onnx_path = Path("onnx")

task = "feature-extraction"
# create ORTOptimizer and define optimization configuration
optimizer = ORTOptimizer.from_pretrained(model_id, feature=task)
optimization_config = OptimizationConfig(optimization_level=99) # enable all optimizations

# apply the optimization configuration to the model
optimizer.export(
    onnx_model_path=onnx_path / "model.onnx",
    onnx_optimized_model_output_path=onnx_path / "model-optimized.onnx",
    optimization_config=optimization_config,
)

KeyError: "Unknown task: feature-extraction. Possible values are [<class 'transformers.models.auto.modeling_auto.AutoModel'>, <class 'transformers.models.auto.modeling_auto.AutoModelForMaskedLM'>, <class 'transformers.models.auto.modeling_auto.AutoModelForCausalLM'>, <class 'transformers.models.auto.modeling_auto.AutoModelForSeq2SeqLM'>, <class 'transformers.models.auto.modeling_auto.AutoModelForSequenceClassification'>, <class 'transformers.models.auto.modeling_auto.AutoModelForTokenClassification'>, <class 'transformers.models.auto.modeling_auto.AutoModelForMultipleChoice'>, <class 'transformers.models.auto.modeling_auto.AutoModelForQuestionAnswering'>, <class 'transformers.models.auto.modeling_auto.AutoModelForImageClassification'>, <class 'transformers.models.auto.modeling_auto.AutoModelForMaskedImageModeling'>]"

In [5]:
%%time
prediction = optimum_feature_extractor("What's my name?")

CPU times: user 131 ms, sys: 2.26 ms, total: 133 ms
Wall time: 120 ms


## Question Answering

### Base Model

In [6]:
from transformers import AutoTokenizer, pipeline

model_id = "deepset/roberta-base-squad2"
task = "question-answering"

tokenizer = AutoTokenizer.from_pretrained(model_id)

# test the model with using transformers pipeline, with handle_impossible_answer for squad_v2 
qa_model = pipeline(task, model=model_id, tokenizer=tokenizer, handle_impossible_answer=True)

In [7]:
%%time
prediction = qa_model(question="what is my name?", context="My name is Humza and I'm 5 years old")
print(prediction)

{'score': 0.6795036196708679, 'start': 11, 'end': 16, 'answer': 'Humza'}
CPU times: user 2.66 s, sys: 0 ns, total: 2.66 s
Wall time: 1.83 s


### Optimized Model

In [8]:
from pathlib import Path
from transformers import AutoTokenizer, pipeline
from optimum.onnxruntime import ORTModelForQuestionAnswering

model_id = "deepset/roberta-base-squad2"
onnx_path = Path("onnx_qa")
task = "question-answering"

# load vanilla transformers and convert to onnx
model = ORTModelForQuestionAnswering.from_pretrained(model_id, from_transformers=True)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# save onnx checkpoint and tokenizer
model.save_pretrained(onnx_path)
tokenizer.save_pretrained(onnx_path)

Downloading: 100%|██████████| 571/571 [00:00<00:00, 32.5kB/s]


('onnx_qa/tokenizer_config.json',
 'onnx_qa/special_tokens_map.json',
 'onnx_qa/vocab.json',
 'onnx_qa/merges.txt',
 'onnx_qa/added_tokens.json',
 'onnx_qa/tokenizer.json')

In [9]:
from optimum.onnxruntime import ORTOptimizer
from optimum.onnxruntime.configuration import OptimizationConfig

# create ORTOptimizer and define optimization configuration
optimizer = ORTOptimizer.from_pretrained(model_id, feature=task)
optimization_config = OptimizationConfig(optimization_level=99) # enable all optimizations

# apply the optimization configuration to the model
optimizer.export(
    onnx_model_path=onnx_path / "model.onnx",
    onnx_optimized_model_output_path=onnx_path / "model-optimized.onnx",
    optimization_config=optimization_config,
)

RuntimeError: Failed to import optimum.onnxruntime.optimization because of the following error (look up to see its traceback):
No module named 'datasets'

In [ ]:
from optimum.onnxruntime import ORTModelForQuestionAnswering

# load quantized model
opt_model = ORTModelForQuestionAnswering.from_pretrained(onnx_path, file_name="model-optimized.onnx")

# test the quantized model with using transformers pipeline
optimum_qa_model = pipeline(task, model=opt_model, tokenizer=tokenizer, handle_impossible_answer=True)

In [ ]:
%%time
prediction = optimum_qa_model(question="what is my name?", context="My name is Humza and I'm 5 years old")
print(prediction)

### Quantized Model

In [ ]:
from optimum.onnxruntime import ORTQuantizer
from optimum.onnxruntime.configuration import AutoQuantizationConfig

# create ORTQuantizer and define quantization configuration
quantizer = ORTQuantizer.from_pretrained(model_id, feature=task)
qconfig = AutoQuantizationConfig.avx512_vnni(is_static=False, per_channel=True)

# apply the quantization configuration to the model
quantizer.export(
    onnx_model_path=onnx_path / "model-optimized.onnx",
    onnx_quantized_model_output_path=onnx_path / "model-quantized.onnx",
    quantization_config=qconfig,
)

In [ ]:
# load quantized model
qt_model = ORTModelForQuestionAnswering.from_pretrained(onnx_path, file_name="model-quantized.onnx")

# test the quantized model with using transformers pipeline
quantized_qa_model = pipeline(task, model=qt_model, tokenizer=tokenizer, handle_impossible_answer=True)

In [ ]:
%%time
prediction = quantized_qa_model(question="what is my name?", context="My name is Humza and I'm 5 years old")
print(prediction)